In [33]:
import scipy.io
import mat73
import pandas as pd
import numpy as np
from numpy import array
from numpy.random import uniform
from numpy import hstack
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.svm import SVC as svm
from sklearn.linear_model import LogisticRegression as lg
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score,f1_score
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter,filterwarnings

import os
# ignore all future warnings1
simplefilter(action='ignore', category=FutureWarning)

## EEG data:

In [34]:
## data loading
data = mat73.loadmat('x_source.mat')  
id = scipy.io.loadmat('id.mat')
IDs = pd.DataFrame(id["id"])


In [35]:
# flattening
df2 = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))
df2 = np.array(df2)
df2 = df2.reshape((2041,68,391)) 
df2.shape

(2041, 68, 391)

In [36]:
sparsed= np.concatenate([np.expand_dims(df2[:,:,i:i+10].mean(axis = 2), axis = 2) for i in range(0,381,10)], axis = 2)

#sanity check
df2[0,0,10:20].mean()
sparsed[0,0,1]
sparsed.shape

(2041, 68, 39)

In [37]:
#flattening and adding id
df2 = pd.DataFrame(sparsed.reshape((sparsed.shape[0], -1)))
df2.shape

df2['IDs'] = IDs
df2.shape

(2041, 2653)

In [38]:
#df2['IDs']#array of arrays
#df2['IDs'][0][0] #this is what i want for 1 guy
df2['IDs'] = [df2['IDs'][i][0] for i in range(2041)] #removing these weird square brackets frm ids
df2.shape

(2041, 2653)

## Behavioral Data:


In [39]:
behaviour_data = pd.read_csv('scores.csv')

### We need columns with:
1. IDs

 
2. for Anxiety:  "SCARED_SR" "SCARED_P". 
    


2. for ADHD inattention: Strengths and Weaknesses of ADHD Symptoms and Normal Behavior Scale ("SWAN IN")


3. for ADHD combined: Strengths and Weaknesses of ADHD Symptoms and Normal Behavior Scale ("SWAN HY")


4. for Autism Social Responsiveness Scale-2 ("SRS_Pre" (preschool) or "SRS") 


5. for Depression: Children's Depression Index (average score of: "CDI_SR" "CDI_P") (alternative 1: Mood & Feelings Questionnaire (MFQ); Alternative 2: Affective Reactivity Index—(ARI-S) Self Report)


6. WISC-Full IQ "WISC_FSIQ"


In [40]:
behaviour_data = behaviour_data[['IDs', 'MFQ_P_Total', 'SCARED_P_Total', 'SWAN_HY', "SWAN_IN", 'SRS_Total','ARI_S_Total_Score']]#, 'WISC_FSIQ' ]]
behaviour_data.shape

(3076, 7)

In [41]:
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
    '% of Total Values', ascending=False).round(1)
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns (with column 'IDs').\n"      
        "There are " + str(mis_val_table_ren_columns.shape[0]) +
            " columns that have missing values.")
    return mis_val_table_ren_columns

missing_values_table(behaviour_data)

Your selected dataframe has 7 columns (with column 'IDs').
There are 6 columns that have missing values.


,Missing Values,% of Total Values
MFQ_P_Total,466,15.1
SCARED_P_Total,328,10.7
SRS_Total,218,7.1
SWAN_HY,163,5.3
SWAN_IN,163,5.3
ARI_S_Total_Score,125,4.1


## Merge EEG and behavioral data together:

In [42]:
df = pd.merge(df2, behaviour_data, on='IDs', how='inner')
df.shape

(1929, 2659)

## features and labels preparation:

In [43]:
#removing NaNs
df = df.dropna()
df.shape

(1599, 2659)

In [44]:

X = df.iloc[:, 0:2652].to_numpy()

In [45]:
Y = df[[ 'MFQ_P_Total', 'SCARED_P_Total', 'SWAN_HY', "SWAN_IN", 'SRS_Total','ARI_S_Total_Score']]#, 'WISC_FSIQ' ]]

#### Data normalisation:

In [46]:
Y = Y.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
Y_scaled = min_max_scaler.fit_transform(Y)
Y = pd.DataFrame(Y_scaled)
Y.shape




(1599, 6)

In [60]:

Y.mean()

0    0.161708
1    0.210177
2    0.543870
3    0.615419
4    0.301258
5    0.289921
dtype: float64

#### PCA on X dimension, not needed:



In [47]:
# dimensionality reduction
#pca = PCA(.95) # 95% variance retained
#pca.fit(X)

# transform data
#X_pca = pca.transform(X)
#X_pca.shape
#X_pca = X

In [48]:
print("X:", X.shape, "Y:", Y.shape)

X: (1599, 2652) Y: (1599, 6)


In [49]:
in_dim = X.shape[1]
out_dim = Y.shape[1]
xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.15)

## MODELLING PART

### dummy regressors (to obtain the random baseline):

In [74]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, median_absolute_error
from sklearn.dummy import DummyRegressor



xtrain, xtest, ytrain, ytest=train_test_split(X, Y, test_size=0.5)
lm_dummy_mean = DummyRegressor(strategy = 'mean').fit(xtrain, ytrain)
lm_dummy_median = DummyRegressor(strategy = 'median').fit(xtrain, ytrain)
ypred_dummy_mean = lm_dummy_mean.predict(xtest)
ypred_dummy_median = lm_dummy_median.predict(xtest)


print("Mean squared error (dummy): {:.2f}".format(mean_squared_error(ytest,
																	ypred_dummy_mean)))

print("Median absolute error (dummy): {:.2f}".format(median_absolute_error(ytest,
																	ypred_dummy_median)))

print("r2_score (dummy mean): {:.2f}".format(r2_score(ytest, ypred_dummy_mean)))
print("r2_score (dummy median): {:.2f}".format(r2_score(ytest, ypred_dummy_median)))

print(ytest[:10])
print(Y.mean())

Mean squared error (dummy): 0.04
Median absolute error (dummy): 0.12
r2_score (dummy mean): -0.00
r2_score (dummy median): -0.03
             0         1         2         3         4         5
411   0.125000  0.028986  0.666667  0.500000  0.211765  0.166667
122   0.071429  0.086957  0.648148  0.648148  0.252941  0.250000
84    0.017857  0.086957  0.518518  0.685185  0.464706  0.333333
554   0.250000  0.130435  0.759259  0.851852  0.335294  0.500000
1298  0.160714  0.376812  0.537037  0.425926  0.252941  0.166667
1140  0.017857  0.028986  0.629630  0.518518  0.135294  0.166667
1363  0.642857  0.913043  0.351852  0.981481  0.705882  0.083333
389   0.053571  0.028986  0.555555  0.685185  0.100000  0.166667
1316  0.035714  0.000000  0.685185  0.907407  0.176471  0.166667
941   0.196429  0.101449  0.314815  0.333333  0.088235  0.500000
0    0.161708
1    0.210177
2    0.543870
3    0.615419
4    0.301258
5    0.289921
dtype: float64


### SVR regressor

In [72]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Create the SVR regressor
svr = SVR(epsilon=0.2)
#Create the Multioutput Regressor
mor = MultiOutputRegressor(svr)
# Train the regressor
mor = mor.fit(xtrain, ytrain)
# Generate predictions for testing data
ypred = mor.predict(xtest)


print("Mean squared error (SVC): {:.2f}".format(mean_squared_error(ytest,
																	ypred)))



Mean squared error (SVC): 0.04


## keras, simple MLP


In [ ]:
model = Sequential()
model.add(Dense(100, input_dim=in_dim, activation="relu"))
model.add(Dense(32, activation="relu"))
model.add(Dense(out_dim))
model.compile(loss="mse", metrics = ['mse'], optimizer="adam")




model.fit(xtrain, ytrain, epochs=100, batch_size=12, validation_split = 0.2, verbose=1)
ypred = model.predict(xtest)




2021-11-18 07:44:31.732141: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-18 07:44:31.754893: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-18 07:44:31.755565: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:00:06.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-11-18 07:44:31.755806: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-11-18 07:44:31.756981: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-18 07:44:31.758258: I tensorflow/stream_executor/platform/default/d

Epoch 1/100
 1/91 [..............................] - ETA: 0s - loss: 1.1547 - mse: 1.1547

2021-11-18 07:44:32.673106: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


91/91 [==============================] - 0s 3ms/step - loss: 0.0798 - mse: 0.0798 - val_loss: 0.0541 - val_mse: 0.0541
Epoch 2/100
91/91 [==============================] - 0s 2ms/step - loss: 0.0451 - mse: 0.0451 - val_loss: 0.0440 - val_mse: 0.0440
Epoch 3/100
91/91 [==============================] - 0s 2ms/step - loss: 0.0454 - mse: 0.0454 - val_loss: 0.0532 - val_mse: 0.0532
Epoch 4/100
91/91 [==============================] - 0s 2ms/step - loss: 0.0408 - mse: 0.0408 - val_loss: 0.0460 - val_mse: 0.0460
Epoch 5/100
91/91 [==============================] - 0s 2ms/step - loss: 0.0403 - mse: 0.0403 - val_loss: 0.0445 - val_mse: 0.0445
Epoch 6/100
91/91 [==============================] - 0s 2ms/step - loss: 0.0387 - mse: 0.0387 - val_loss: 0.0481 - val_mse: 0.0481
Epoch 7/100
91/91 [==============================] - 0s 2ms/step - loss: 0.0393 - mse: 0.0393 - val_loss: 0.0479 - val_mse: 0.0479
Epoch 8/100
91/91 [==============================] - 0s 2ms/step - loss: 0.0350 - mse: 0.0350 -

In [ ]:
score = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)

score= np.array(score)

score.mean()

0.059276782

In [ ]:
#xtrain = np.asarray(xtrain).astype('float32')
#xtest = np.asarray(xtest).astype('float32')

In [ ]:
from tensorflow.keras.layers import Conv2D , MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
def get_CNN_2_classifier():

    k = 3  # kernel size
    s = 2  # stride
    n_filters = 32 # number of filters
    inputs = Input(shape=(39, 68, 1))

    conv1 = Conv2D(n_filters, kernel_size=(k, k), strides=(s, s), padding='same')(inputs)
    conv1 = BatchNormalization(scale=False, axis=3)(conv1)
    conv1 = LeakyReLU(alpha=0.2)(conv1)
    conv1 = Conv2D(n_filters, kernel_size=(k, k), padding='same')(conv1)
    conv1 = BatchNormalization(scale=False, axis=3)(conv1)
    conv1 = LeakyReLU(alpha=0.1)(conv1)
    pool1 = conv1 # MaxPooling2D(pool_size=(s, s))(conv1)

    conv2 = Conv2D(2 * n_filters, kernel_size=(k, k), strides=(s, s), padding='same')(pool1)
    conv2 = BatchNormalization(scale=False, axis=3)(conv2)
    conv2 = LeakyReLU(alpha=0.3)(conv2)
    conv2 = Conv2D(2 * n_filters, kernel_size=(k, k), padding='same')(conv2)
    conv2 = BatchNormalization(scale=False, axis=3)(conv2)
    conv2 = LeakyReLU(alpha=0.1)(conv2)
    pool2 = MaxPooling2D(pool_size=(s, s))(conv2)

    conv3 = Conv2D(4 * n_filters, kernel_size=(k, k), padding='same')(pool2)
    conv3 = BatchNormalization(scale=False, axis=3)(conv3)
    conv3 = LeakyReLU(alpha=0.1)(conv3)
    conv3 = Conv2D(4 * n_filters, kernel_size=(k, k), padding='same')(conv3)
    conv3 = BatchNormalization(scale=False, axis=3)(conv3)
    conv3 = LeakyReLU(alpha=0.1)(conv3)
    pool3 = MaxPooling2D(pool_size=(s, s))(conv3)

    conv4 = Conv2D(8 * n_filters, kernel_size=(k, k), padding='same')(pool3)
    conv4 = BatchNormalization(scale=False, axis=3)(conv4)
    conv4 = LeakyReLU(alpha=0.1)(conv4)
    conv4 = Conv2D(8 * n_filters, kernel_size=(k, k), padding='same')(conv4)
    conv4 = BatchNormalization(scale=False, axis=3)(conv4)
    conv4 = LeakyReLU(alpha=0.1)(conv4)
    pool4 = MaxPooling2D(pool_size=(s, s))(conv4)

    conv5 = Conv2D(16 * n_filters, kernel_size=(k, k), padding='same')(pool4)
    conv5 = BatchNormalization(scale=False, axis=3)(conv5)
    conv5 = LeakyReLU(alpha=0.1)(conv5)
    conv5 = Conv2D(16 * n_filters, kernel_size=(k, k), padding='same')(conv5)
    conv5 = BatchNormalization(scale=False, axis=3)(conv5)
    conv5 = LeakyReLU(alpha=0.1)(conv5)

    gap = GlobalAveragePooling2D()(conv5)
    outputs = Dense(6, activation='relu')(gap)

    model = Model(inputs, outputs)
    model.compile(optimizer = Adam(lr=1e-4), loss = 'mse', metrics = ['mse'])
    model.summary()
  
    return model

CNN_model_2 = get_CNN_2_classifier()


In [75]:

def get_CNN_1_classifier():

    k = 3  # kernel size
    s = 2  # stride
    n_filters = 32 # number of filters
    inputs = Input(shape=(39, 68, 1))

    conv1 = Conv2D(n_filters, kernel_size=(k, k), strides=(s, s), padding='same')(inputs)
    conv1 = BatchNormalization(scale=False, axis=3)(conv1)
    conv1 = LeakyReLU(alpha=0.2)(conv1)
    conv1 = Conv2D(n_filters, kernel_size=(k, k), padding='same')(conv1)
    conv1 = BatchNormalization(scale=False, axis=3)(conv1)
    conv1 = LeakyReLU(alpha=0.1)(conv1)
    pool1 = conv1 # MaxPooling2D(pool_size=(s, s))(conv1)


    conv4 = Conv2D(8 * n_filters, kernel_size=(k, k), padding='same')(pool1)
    conv4 = BatchNormalization(scale=False, axis=3)(conv4)
    conv4 = LeakyReLU(alpha=0.1)(conv4)
    conv4 = Conv2D(8 * n_filters, kernel_size=(k, k), padding='same')(conv4)
    conv4 = BatchNormalization(scale=False, axis=3)(conv4)
    conv4 = LeakyReLU(alpha=0.1)(conv4)
    pool4 = MaxPooling2D(pool_size=(s, s))(conv4)

    conv5 = Conv2D(16 * n_filters, kernel_size=(k, k), padding='same')(pool4)
    conv5 = BatchNormalization(scale=False, axis=3)(conv5)
    conv5 = LeakyReLU(alpha=0.1)(conv5)
    conv5 = Conv2D(16 * n_filters, kernel_size=(k, k), padding='same')(conv5)
    conv5 = BatchNormalization(scale=False, axis=3)(conv5)
    conv5 = LeakyReLU(alpha=0.1)(conv5)

    gap = GlobalAveragePooling2D()(conv5)
    outputs = Dense(6, activation='relu')(gap)

    model = Model(inputs, outputs)
    model.compile(optimizer = Adam(lr=1e-4), loss = 'mse', metrics = ['mse'])
    model.summary()
  
    return model

CNN_model_1 = get_CNN_1_classifier()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 39, 68, 1)]       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 20, 34, 32)        320       
_________________________________________________________________
batch_normalization_10 (Batc (None, 20, 34, 32)        96        
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 20, 34, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 20, 34, 32)        9248      
_________________________________________________________________
batch_normalization_11 (Batc (None, 20, 34, 32)        96        
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 20, 34, 32)        0   

In [81]:
# data reshaping for CNN
n_electrode = 68
n_freq =39 

xtrain = xtrain.reshape((xtrain.shape[0], n_electrode, n_freq))
xtest = xtest.reshape((xtest.shape[0], n_electrode, n_freq))



In [77]:
history_CNN_1 = CNN_model_1.fit(np.transpose(xtrain, (0, 2, 1)), ytrain, validation_split=0.2, epochs=50, verbose=1)


ypred = CNN_model_1.predict(np.transpose(xtest, (0, 2, 1)))



score = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)

score = np.array(score)

score.mean()

Epoch 1/50
20/20 [==============================] - 2s 94ms/step - loss: 0.1533 - mse: 0.1533 - val_loss: 0.1773 - val_mse: 0.1773
Epoch 2/50
20/20 [==============================] - 1s 36ms/step - loss: 0.1044 - mse: 0.1044 - val_loss: 0.1762 - val_mse: 0.1762
Epoch 3/50
20/20 [==============================] - 1s 37ms/step - loss: 0.0871 - mse: 0.0871 - val_loss: 0.1366 - val_mse: 0.1366
Epoch 4/50
20/20 [==============================] - 1s 37ms/step - loss: 0.0699 - mse: 0.0699 - val_loss: 0.1669 - val_mse: 0.1669
Epoch 5/50
20/20 [==============================] - 1s 36ms/step - loss: 0.0497 - mse: 0.0497 - val_loss: 0.1153 - val_mse: 0.1153
Epoch 6/50
20/20 [==============================] - 1s 37ms/step - loss: 0.0397 - mse: 0.0397 - val_loss: 0.1117 - val_mse: 0.1117
Epoch 7/50
20/20 [==============================] - 1s 37ms/step - loss: 0.0351 - mse: 0.0351 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 8/50
20/20 [==============================] - 1s 37ms/step - loss: 0.0326 - m

0.060835075

In [ ]:
history_CNN_2 = CNN_model_2.fit(np.transpose(xtrain, (0, 2, 1)), ytrain, validation_split=0.2, epochs=50, verbose=1)


ypred = CNN_model_2.predict(np.transpose(xtest, (0, 2, 1)))



score = tf.keras.metrics.mean_squared_error(
    ytest, ypred
)

score = np.array(score)

score.mean()


Epoch 1/50
34/34 [==============================] - 0s 13ms/step - loss: 0.0029 - mse: 0.0029 - val_loss: 0.0507 - val_mse: 0.0507
Epoch 2/50
34/34 [==============================] - 0s 11ms/step - loss: 0.0026 - mse: 0.0026 - val_loss: 0.0506 - val_mse: 0.0506
Epoch 3/50
34/34 [==============================] - 0s 11ms/step - loss: 0.0027 - mse: 0.0027 - val_loss: 0.0498 - val_mse: 0.0498
Epoch 4/50
34/34 [==============================] - 0s 11ms/step - loss: 0.0026 - mse: 0.0026 - val_loss: 0.0496 - val_mse: 0.0496
Epoch 5/50
34/34 [==============================] - 0s 11ms/step - loss: 0.0026 - mse: 0.0026 - val_loss: 0.0508 - val_mse: 0.0508
Epoch 6/50
34/34 [==============================] - 0s 11ms/step - loss: 0.0026 - mse: 0.0026 - val_loss: 0.0492 - val_mse: 0.0492
Epoch 7/50
34/34 [==============================] - 0s 11ms/step - loss: 0.0031 - mse: 0.0031 - val_loss: 0.0505 - val_mse: 0.0505
Epoch 8/50
34/34 [==============================] - 0s 11ms/step - loss: 0.0027 - m

0.03990904